# Kahneman–Tversky Optimization (KTO) at Scale with LoRA

This guide provides a step-by-step workflow for preference fine-tuning the `meta-llama/Meta-Llama-3-8B-Instruct` model on a multi-GPU Anyscale cluster. We’ll use **LLaMA-Factory** as the training framework and **LoRA** to reduce memory footprint and enable efficient multi-GPU training.

**What is KTO?** *Kahneman–Tversky Optimization* aligns a model to human preferences using **single binary labels (accept/reject)** instead of pairwise “chosen vs. rejected” comparisons. KTO directly optimizes the policy on these unary signals, simplifying data preparation while still encouraging preferred behavior and discouraging undesired outputs.

## Step 1: Set Up Your Environment
### Dependencies
First, we need to ensure our environment has the right libraries. We'll start with a pre-built container image and install LLaMA-Factory and DeepSpeed on top of it.

Recommended Container Image:
```bash
anyscale/ray-llm:2.48.0-py311-cu128
```

Execute the following commands to install the required packages and optional tools for experiment tracking and faster downloads.

In [1]:
%%bash
# Install the specific version of LLaMA-Factory
pip install -q llamafactory@git+https://github.com/hiyouga/LLaMA-Factory.git@v0.9.3

# (Optional) For accelerated model downloads from Hugging Face
pip install -q hf_transfer==0.1.9

# (Optional) Acceleration methods (ensure CUDA/Torch compatibility)
pip install -q flash-attn==2.8.3 liger-kernel==0.6.2

# (Optional) Experiment tracking library
pip install -q mlflow==3.4.0

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-server 1.24.0 requires anyio<4,>=3.1.0, but you have anyio 4.10.0 which is incompatible.


Successfully registered `llamafactory` package to be installed on all cluster nodes.
View and update dependencies here: https://console.anyscale.com/cld_kvedZWag2qA8i5BjxUevf5i7/prj_cz951f43jjdybtzkx1s5sjgz99/workspaces/expwrk_28b5ivx1sj8wei1sfv965t9fqr?workspace-tab=dependencies
Successfully registered `hf_transfer` package to be installed on all cluster nodes.
View and update dependencies here: https://console.anyscale.com/cld_kvedZWag2qA8i5BjxUevf5i7/prj_cz951f43jjdybtzkx1s5sjgz99/workspaces/expwrk_28b5ivx1sj8wei1sfv965t9fqr?workspace-tab=dependencies
Successfully registered `flash-attn, liger-kernel` packages to be installed on all cluster nodes.
View and update dependencies here: https://console.anyscale.com/cld_kvedZWag2qA8i5BjxUevf5i7/prj_cz951f43jjdybtzkx1s5sjgz99/workspaces/expwrk_28b5ivx1sj8wei1sfv965t9fqr?workspace-tab=dependencies


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llamafactory 0.9.3 requires pydantic<=2.10.6, but you have pydantic 2.11.9 which is incompatible.


Successfully registered `mlflow` package to be installed on all cluster nodes.
View and update dependencies here: https://console.anyscale.com/cld_kvedZWag2qA8i5BjxUevf5i7/prj_cz951f43jjdybtzkx1s5sjgz99/workspaces/expwrk_28b5ivx1sj8wei1sfv965t9fqr?workspace-tab=dependencies


## Model and Resources

| Item | Value |
|------|-------|
| **Base model** | [`meta-llama/Meta-Llama-3-8B-Instruct`](https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct) |
| **Workers** | 4 × L40S / A100 (1 GPU each) |

> Compared to SFT, KTO typically holds two copies of the model (policy + reference), and alignment datasets often use long contexts, so GPUs with larger VRAM are recommended. Techniques like **LoRA** and memory-efficient attention can further reduce memory pressure.

## Step 2: Prepare the Dataset

### Understand the Dataset
For this tutorial, we will use `kto_en_demo`, a unary-preference dataset for **KTO (Kahneman–Tversky Optimization)**.  
Each record contains a multi-turn ShareGPT-style dialogue plus a **binary label** indicating whether the modeled behavior is preferred.

This dataset contains:
- `messages`: Turn-by-turn chat between a user and the assistant.
- `label`: A boolean (`true`/`false`) indicating whether the example is preferred.

**Note:** To maintain role alignment in ShareGPT format, a strict turn order must be followed: `human` and `observation` (tool output) must appear in odd-numbered positions (1, 3, 5, ...), while `gpt` and `function_call` must appear in even-numbered positions (2, 4, 6, ...). The model learns to generate the content in the `gpt` and `function_call` turns.

<details>
  <summary>Dataset Example</summary>

  ```json
  {
    "messages": [
      { "role": "user", "content": "Compare and contrast the roles of the hippocampus and the prefrontal cortex..." },
      { "role": "assistant", "content": "The human brain is a highly complex organ, responsible for a myriad of cognitive functions..." },
      { "role": "user", "content": "Discuss the mechanisms through which the prefrontal cortex ..." },
      { "role": "assistant", "content": "The prefrontal cortex (PFC)..." },
      { "role": "user", "content": "Can you elaborate on the role of the amygdala..." },
      { "role": "assistant", "content": "The amygdala plays a crucial role in the emotional processing of stored memories..." }
    ],
    "label": true
  }
  ```

</details>

### Register the local dataset

To specify new datasets that are accessible across Ray worker nodes, add all dataset files and a `dataset_info.json` to **[storage shared across nodes](https://docs.anyscale.com/configuration/storage#shared)** such as `/mnt/cluster_storage`.

For example, to run KTO fine-tuning on `kto_en_demo` locally:

`dataset_info.json`

- `kto_tag` maps the unary preference label used by KTO.
- `tags` helps the loader interpret role/content fields in ShareGPT-style records.

```json
{
  "my_kto_en_demo": {
    "file_name": "/mnt/cluster_storage/kto_en_demo.json",
    "formatting": "sharegpt",
    "columns": {
      "messages": "messages",
      "kto_tag": "label"
    },
    "tags": {
      "role_tag": "role",
      "content_tag": "content",
      "user_tag": "user",
      "assistant_tag": "assistant"
    }
  }
}
```

> For a more detailed dataset preparation and formatting guide, follow [_](https://docs.anyscale.com/llm/fine-tuning/data-preparation#data-format).


In [2]:
%%bash
# Make sure all files are accessible to worker nodes
# Create a copy of the data in /mnt/cluster_storage
wget https://anyscale-public-materials.s3.us-west-2.amazonaws.com/llm-finetuning/llama-factory/datasets/sharegpt/kto_en_demo.json -O /mnt/cluster_storage/kto_en_demo.json
# Create a copy of the dataset registry in /mnt/cluster_storage
cp ../dataset-configs/dataset_info.json /mnt/cluster_storage/

--2025-09-22 17:31:23--  https://anyscale-public-materials.s3.us-west-2.amazonaws.com/llm-finetuning/llama-factory/datasets/sharegpt/kto_en_demo.json
Resolving anyscale-public-materials.s3.us-west-2.amazonaws.com (anyscale-public-materials.s3.us-west-2.amazonaws.com)... 3.5.81.39, 52.218.152.49, 52.92.137.194, ...
Connecting to anyscale-public-materials.s3.us-west-2.amazonaws.com (anyscale-public-materials.s3.us-west-2.amazonaws.com)|3.5.81.39|:443... connected.


HTTP request sent, awaiting response... 200 OK
Length: 913519 (892K) [application/json]
Saving to: ‘/mnt/cluster_storage/kto_en_demo.json’

     0K .......... .......... .......... .......... ..........  5%  222M 0s
    50K .......... .......... .......... .......... .......... 11% 38.3M 0s
   100K .......... .......... .......... .......... .......... 16% 72.7M 0s
   150K .......... .......... .......... .......... .......... 22%  261M 0s
   200K .......... .......... .......... .......... .......... 28% 56.6M 0s
   250K .......... .......... .......... .......... .......... 33%  259M 0s
   300K .......... .......... .......... .......... .......... 39%  248M 0s
   350K .......... .......... .......... .......... .......... 44%  269M 0s
   400K .......... .......... .......... .......... .......... 50%  257M 0s
   450K .......... .......... .......... .......... .......... 56%  249M 0s
   500K .......... .......... .......... .......... .......... 61%  266M 0s
   550K .......... .....

## Step 3: Create the Preference-Tuning Config (KTO + LoRA)

Create a YAML file that defines your **KTO** run. It specifies the base model, dataset, **LoRA** settings, KTO hyperparameters, optional acceleration methods, logging, and Ray cluster resources.

**Important notes:**
- **Acceleration libs:** `flash_attn` and `liger-kernel` can be used together, but actual speed and memory gains vary with GPU architecture, sequence length, batch size, precision, kernel availability. Benchmark your training workloads to confirm improvements. Note that `fa2` is not supported on Turing GPUs (e.g., T4).
- **Access & paths:** The YAML only needs to be on the **head node**, but any referenced paths (e.g., `dataset_dir`, `ray_storage_path`, `output_dir`) must be on **shared storage** (such as `/mnt/cluster_storage/`) visible to all workers.
- **Gated models:** If your base is gated, set `HF_TOKEN` in the runtime env.
- **Memory tips:** If VRAM is tight, consider enabling gradient checkpointing or switching to QLoRA (4/8-bit), then add the corresponding quantization keys.

### LLaMA-Factory + Ray Configuration

```yaml
# train-configs/kto_lora.yaml

### model
model_name_or_path: meta-llama/Meta-Llama-3-8B-Instruct
trust_remote_code: true

### method
stage: kto
do_train: true
finetuning_type: lora
lora_rank: 8
lora_target: all
pref_beta: 0.1

### acceleration methods
# both methods can be enabled at the same time
flash_attn: fa2            # speed up attention and cut activation memory at long context, use auto on Turing GPUs (e.g., T4)
enable_liger_kernel: true  # reduce VRAM and improve throughput across multiple transformer ops

### dataset
dataset: my_kto_en_demo
dataset_dir: /mnt/cluster_storage

template: llama3
cutoff_len: 1024
max_samples: 1000
overwrite_cache: true
preprocessing_num_workers: 16

### output
output_dir: llama3_8b_lora_kto
logging_steps: 5
save_steps: 50
plot_loss: true
overwrite_output_dir: true
report_to: mlflow   # or none

### train
per_device_train_batch_size: 1
gradient_accumulation_steps: 2
num_train_epochs: 3.0  # low for demo purpose; adjust as needed
learning_rate: 5.0e-6
bf16: true
lr_scheduler_type: cosine
warmup_ratio: 0.1
ddp_timeout: 180000000

### ray
ray_run_name: llama3_8b_kto_lora
ray_storage_path: /mnt/cluster_storage/
ray_num_workers: 4
resources_per_worker:
  GPU: 1
  anyscale/accelerator_shape:4xL40S: 0.001  # pin a specific node shape
  # accelerator_type:L40S: 0.001            # or just request a GPU type

ray_init_kwargs:
  runtime_env:
    env_vars:
      # if using gated models like meta-llama/Llama-3-8B-Instruct
      HF_TOKEN: <your_huggingface_token>
      # Enable faster downloads if hf_transfer is installed:
      HF_HUB_ENABLE_HF_TRANSFER: '1'
      # if using mlflow for experiments tracking
      MLFLOW_TRACKING_URI: "https://<your_cloud_id>.cloud.databricks.com"
      MLFLOW_TRACKING_TOKEN: "<mlflow_tracking_token>"
      MLFLOW_EXPERIMENT_NAME: "/Users/<your_user_id>/experiment_name"
```

## Step 4: Train and Monitor

With all configuration in place, you can launch fine-tuning/post-training in one of two ways.

### Option A — Run from a Workspace (quick start)

The `USE_RAY=1` prefix tells LLaMA-Factory to run in distributed mode on the Ray cluster attached to your workspace.

In [ ]:
%%bash
USE_RAY=1 llamafactory-cli train ../train-configs/kto_lora.yaml

INFO 09-22 17:58:43 [__init__.py:248] No platform detected, vLLM is running on UnspecifiedPlatform
WARNING 09-22 17:58:43 [_custom_ops.py:20] Failed to import from vllm._C with ImportError('libcuda.so.1: cannot open shared object file: No such file or directory')


2025-09-22 17:58:47,161	INFO worker.py:1747 -- Connecting to existing Ray cluster at address: 10.0.84.195:6379...
2025-09-22 17:58:47,173	INFO worker.py:1918 -- Connected to Ray cluster. View the dashboard at https://session-zwu6m5dfjub67c3wfgsrmyfc7f.i.anyscaleuserdata.com 
2025-09-22 17:58:47,175	INFO packaging.py:380 -- Pushing file package 'gcs://_ray_pkg_bfae09dd86bdebe0b1c521da35d6b3a9fcf596f8.zip' (0.22MiB) to Ray cluster...
2025-09-22 17:58:47,176	INFO packaging.py:393 -- Successfully pushed file package 'gcs://_ray_pkg_bfae09dd86bdebe0b1c521da35d6b3a9fcf596f8.zip'.



View detailed results here: /mnt/cluster_storage/llama3_8b_kto_lora
To visualize your results with TensorBoard, run: `tensorboard --logdir /tmp/ray/session_2025-09-22_17-27-35_971622_2432/artifacts/2025-09-22_17-58-47/llama3_8b_kto_lora/driver_artifacts`

Training started with configuration:
╭──────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Training config                                                                                              │
├──────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ train_loop_config/args/bf16                                                                             True │
│ train_loop_config/args/cutoff_len                                                                       1024 │
│ train_loop_config/args/dataset                                                                my_kto_en_demo │
│ train_loop_config/args/dat

(RayTrainWorker pid=9270, ip=10.0.121.215) Setting up process group for: env:// [rank=0, world_size=4]
(TorchTrainer pid=9165, ip=10.0.121.215) Started distributed worker processes: 
(TorchTrainer pid=9165, ip=10.0.121.215) - (node_id=a82307d5edc33319835b52f5f6a1b9248a325214f61311ca3dee4da5, ip=10.0.121.215, pid=9270) world_rank=0, local_rank=0, node_rank=0
(TorchTrainer pid=9165, ip=10.0.121.215) - (node_id=a82307d5edc33319835b52f5f6a1b9248a325214f61311ca3dee4da5, ip=10.0.121.215, pid=9272) world_rank=1, local_rank=1, node_rank=0
(TorchTrainer pid=9165, ip=10.0.121.215) - (node_id=a82307d5edc33319835b52f5f6a1b9248a325214f61311ca3dee4da5, ip=10.0.121.215, pid=9271) world_rank=2, local_rank=2, node_rank=0
(TorchTrainer pid=9165, ip=10.0.121.215) - (node_id=a82307d5edc33319835b52f5f6a1b9248a325214f61311ca3dee4da5, ip=10.0.121.215, pid=9269) world_rank=3, local_rank=3, node_rank=0


(RayTrainWorker pid=9270, ip=10.0.121.215) [INFO|2025-09-22 17:58:59] llamafactory.hparams.parser:143 >> Set `ddp_find_unused_parameters` to False in DDP training since LoRA is enabled.
(RayTrainWorker pid=9270, ip=10.0.121.215) [INFO|2025-09-22 17:58:59] llamafactory.hparams.parser:406 >> Process rank: 0, world size: 4, device: cuda:0, distributed training: True, compute dtype: torch.bfloat16


...

(RayTrainWorker pid=9270, ip=10.0.121.215) ***** train metrics *****
(RayTrainWorker pid=9270, ip=10.0.121.215)   epoch                    =        3.0
(RayTrainWorker pid=9270, ip=10.0.121.215)   total_flos               = 19610724GF
(RayTrainWorker pid=9270, ip=10.0.121.215)   train_loss               =     0.4829
(RayTrainWorker pid=9270, ip=10.0.121.215)   train_runtime            = 0:02:47.59
(RayTrainWorker pid=9270, ip=10.0.121.215)   train_samples_per_second =       5.37
(RayTrainWorker pid=9270, ip=10.0.121.215)   train_steps_per_second   =       0.68
(RayTrainWorker pid=9270, ip=10.0.121.215) Figure saved at: llama3_8b_lora_kto/training_loss.png
(RayTrainWorker pid=9270, ip=10.0.121.215) Figure saved at: llama3_8b_lora_kto/training_rewards_chosen.png
(RayTrainWorker pid=9270, ip=10.0.121.215) [WARNING|2025-09-22 18:02:00] llamafactory.extras.ploting:148 >> No metric eval_loss to plot.


(RayTrainWorker pid=9270, ip=10.0.121.215) [INFO|modelcard.py:450] 2025-09-22 18:02:00,536 >> Dropping the following result as it does not have all the necessary fields:
(RayTrainWorker pid=9270, ip=10.0.121.215) {'task': {'name': 'Causal Language Modeling', 'type': 'text-generation'}}



Training completed after 3 iterations at 2025-09-22 18:02:02. Total running time: 3min 14s


2025-09-22 18:02:02,105	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/mnt/cluster_storage/llama3_8b_kto_lora' in 0.0222s.


(RayTrainWorker pid=9270, ip=10.0.121.215) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/mnt/cluster_storage/llama3_8b_kto_lora/TorchTrainer_75e12_00000_0_2025-09-22_17-58-47/checkpoint_000002) [repeated 3x across cluster]


### Option B — Run as an Anyscale Job (production)

For longer or production runs, submit the training as an **Anyscale Job**. Jobs run outside your interactive session for better stability, retries, and durable logs. You’ll package LLaMA-Factory and other libraries in a container image and launch with a short job config. See [_](https://docs.anyscale.com/llm/fine-tuning/llamafactory-jobs) for the step-by-step guide.

### Tracking with MLflow

If you set `report_to: mlflow` in your YAML, LLaMA-Factory will log metrics (loss, learning rate, etc.), parameters, and artifacts to your configured MLflow tracking server.

* **Install MLflow:**

  ```bash
  pip install mlflow
  ```

**Example YAML snippet:**

```yaml
report_to: mlflow

ray_init_kwargs:
  runtime_env:
    env_vars:
      MLFLOW_TRACKING_URI: "https://<your_cloud_id>.cloud.databricks.com"
      MLFLOW_TRACKING_TOKEN: "<mlflow_tracking_token>"
      MLFLOW_EXPERIMENT_NAME: "/Users/<your_user_id>/experiment_name"
```

**MLFlow**
![MLFlow](https://anyscale-public-materials.s3.us-west-2.amazonaws.com/llm-finetuning/llama-factory/3.2.3/3.2.3-mlflow.png)

For a more detailed guide on tracking experiments with other tools such as WandB or TensorBoard, see [_](https://docs.anyscale.com/llm/fine-tuning/observability-and-tracking.md).


## Step 5: Locate Checkpoints

Checkpoints are written under `ray_storage_path/ray_run_name`. In this example run, the path is: `/mnt/cluster_storage/llama3_8b_kto_lora`. 

Inside, you’ll see a **trainer session** directory named like:
`TorchTrainer_75e12_00000_0_2025-09-22_17-58-47`.

- `TorchTrainer_*` is created **when the trainer starts**; the suffix encodes a short run id and the **start timestamp**.
- Within that directory, checkpoints are named `checkpoint_000xxx/`, where the number is the saved ordered checkpoints. 

The save cadence is controlled by `save_strategy` and `save_steps`. For instructions on how to resume interrupted training via `resume_from_checkpoint` and more, see [_](https://docs.anyscale.com/llm/fine-tuning/checkpointing#artifacts-directory).

## Step 6: Export the Model

If you use LoRA, you can keep the base model and adapter separate ([_](https://docs.anyscale.com/llm/serving/multi-lora)) or merge the adapter into the base model for low-latency inference. 

For full fine-tuning or freeze-tuning, export the fine-tuned model directly.

You may optionally apply post-training quantization on merged or full models before serving. See [_](https://docs.anyscale.com/llm/fine-tuning/observability-and-tracking.md#ptq) for the exact export commands and options.